In [1]:
%load_ext autoreload
%autoreload 2

import copy
import os
import sys
import time
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pickle
import sklearn

import torch

torch.manual_seed(0)

import torchvision
from torchvision import transforms
import torch.nn as nn

sys.path.append('./../')
from src.dataset import VehiclePredictorDataset
from src.utils import train_model, evaluate_model, get_model, visualize_model
#from src.bilinear import BCNN

## GPU Check

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## Config Variables

In [3]:
root_path = './../'
data_path = os.path.join(root_path, 'data')
dataset_path = os.path.join(data_path, 'VMMRdb')

# modify this line if you want to train a model on top 100/top 200/top 300 most common make_models
with open(os.path.join(data_path, 'make_model_year_most_common_400.pkl'), 'rb') as f:
    target_make_model_year_labels = pickle.load(f)

# load the dataset for some stats
vp_dataset = VehiclePredictorDataset(root_dir=dataset_path, target_make_model_year_labels=target_make_model_year_labels)
num_images = len(vp_dataset)
num_labels = len(vp_dataset.make_model_year_counts)
class_distribution = vp_dataset.make_model_year_counts
print(f"num_images: {num_images}")
print(f"num_labels: {num_labels}")

num_images: 99562
num_labels: 400


## Define the transforms
- Add other transforms here later, if needed. 
- Do we need any specific transforms for train and val?

In [4]:
def get_transform():
    
    transform = transforms.Compose([
        #transforms.Resize((224, 224)),
        #transforms.Resize((448, 448)),
        
        torchvision.transforms.Resize(size=224),
        transforms.RandomHorizontalFlip(),
        torchvision.transforms.CenterCrop(size=224),
        
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return transform

## Instantiate the train, val and test dataloaders

In [5]:
train_split_ratio = 0.8
val_split_ratio = 0.1
test_split_ration = 0.1

# later see if you can have train-specific transforms
dataset = VehiclePredictorDataset(root_dir=dataset_path, target_make_model_year_labels=target_make_model_year_labels, transform=get_transform())

# split dataset in train and val set
dataset_len = len(dataset)
indices = torch.randperm(dataset_len).tolist()
train_split_index = int(train_split_ratio * dataset_len)
val_split_index = train_split_index + int(val_split_ratio * dataset_len)
train_dataset = torch.utils.data.Subset(dataset, indices[0:train_split_index])
val_dataset = torch.utils.data.Subset(dataset, indices[train_split_index:val_split_index])
test_dataset = torch.utils.data.Subset(dataset, indices[val_split_index:])

set_num_workers = 16;
batch_size = 64

# define the dataloaders
dataloader_train = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=set_num_workers,
)

dataloader_val = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=set_num_workers,
)

dataloader_test = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=set_num_workers,
)

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(val_dataset)
}

print(f'train: 0 to {train_split_index}\nval: {train_split_index} to {val_split_index}\ntest: {val_split_index} to {dataset_len-1}')
print(f'dataset_sizes : {dataset_sizes}')

train: 0 to 79649
val: 79649 to 89605
test: 89605 to 99561
dataset_sizes : {'train': 79649, 'val': 9956, 'test': 9956}


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## Model config

In [6]:
# check https://pytorch.org/vision/0.8/models.html to experiment with other backbone model
#backbone_model = 'bilinear_resnet50_fc'
backbone_model = 'bilinear_vgg16_fc'
num_epochs = 55
lr = 1

# get the model
model = get_model(num_labels, backbone_model).to(device)

# the reason for computing weights is to account for the class imbalance
weight_distribution = 1 / torch.tensor(list(class_distribution.values()))
weight_distribution = weight_distribution / weight_distribution.sum()
weight_distribution = weight_distribution.to(device)

# define the loss function
criterion = nn.CrossEntropyLoss(weight=weight_distribution)

# define the optimizer
optimizer = torch.optim.SGD(model.fc.parameters(), lr, momentum=0.9, weight_decay = 1e-8)

# define the scheduler
#exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)

target_labels = 'makemodelyear'
model_info = {
    'backbone': backbone_model,
    'num_classes': f"{num_labels}classes",
    'num_epochs': f"{num_epochs}epochs",
    'lr': f"{lr}lr",
    'labels': target_labels
}
model_name = "_".join([model_info[key] for key in model_info]) + ".pth"
print(f"model_name: {model_name}")

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
./../src/bilinear_vgg16.py:21: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  torch.nn.init.kaiming_normal(self.fc.weight.data)  # Initialize the fc layers.
./../src/bilinear_vgg16.py:23: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  to

model_name: bilinear_vgg16_fc_400classes_55epochs_1lr_makemodelyear.pth


## Train
- Uncomment this line if you want to train again. 
- The trained models are all stored in this [drive folder]( https://drive.google.com/drive/folders/1RXaKgStTFnVRaLvk-eIHEGhwvQp-y1-c?usp=sharing)

In [7]:
train_model(model, model_info, dataset_sizes, dataloader_train, dataloader_val, criterion, optimizer, exp_lr_scheduler, target_labels, num_epochs)

Epoch 0/54
----------


100%|███████████████████████████████████████████████████████████████████████████████████████████| 1245/1245 [02:26<00:00,  8.51it/s]


train Loss: 5.6222 Acc: 0.0295 Time: 2m 26s


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 156/156 [00:18<00:00,  8.30it/s]


val Loss: 5.2610 Acc: 0.0556 Time: 3m 45s
learning rate: 1
Epoch 1/54
----------


...

val Loss: 2.2091 Acc: 0.3834 Time: 147m 19s
learning rate: 1
Epoch 54/54
----------


100%|███████████████████████████████████████████████████████████████████████████████████████████| 1245/1245 [02:25<00:00,  8.57it/s]


train Loss: 1.0798 Acc: 0.8069 Time: 150m 44s


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 156/156 [00:19<00:00,  7.95it/s]


val Loss: 2.2002 Acc: 0.3830 Time: 150m 4s
learning rate: 1
Training complete in 150m 4s
Best val Acc: 0.384291


## Evaluate on test set

In [9]:
# Load saved model
model_name = 'bilinear_vgg16_fc_400classes_55epochs_1lr_makemodelyear_acc_0.384.pth'
backbone_model = 'bilinear_vgg16_fc'
target_labels = 'makemodelyear'
model = get_model(num_labels, backbone_model).to(device)
model.load_state_dict(torch.load(os.path.join(root_path, 'models', model_name)))

cm, cr = evaluate_model(model, dataset_sizes, dataloader_test, target_labels, target_make_model_year_labels)

# plot the CM
cm_pd = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm_pd = pd.DataFrame(cm_pd, index=target_make_model_year_labels, columns=target_make_model_year_labels)
cr_pd = pd.DataFrame(cr).transpose()

100%|██████████████████████████████████████████████████████████████████████████████████████████| 9957/9957 [01:01<00:00, 162.31it/s]

Test Acc: 0.3840
Average time per image: 0.0062 seconds


In [ ]:
model

- Confusion matrix

In [ ]:
cm_pd

- Classification Report

In [ ]:
cr_pd

- Save/Load the metrics

In [ ]:
cm_pd.to_csv(os.path.join(root_path, 'results', 'bilinear', f"confusion_matrix_{model_name}.csv"), index=False)
cr_pd.to_csv(os.path.join(root_path, 'results', 'bilinear', f"classification_report_{model_name}.csv"), index=False)
#cm_pd_df = pd.read_csv(os.path.join(root_path, 'results', 'baseline_methods', f"confusion_matrix_{model_name}.csv"))
#cr_pd_df = pd.read_csv(os.path.join(root_path, 'results', 'baseline_methods', f"classification_report_{model_name}.csv"))

## Visually Inspect performance
- Load a target model to inpect

In [ ]:
# Load saved model
model = get_model(num_labels, backbone_model).to(device)
model.load_state_dict(torch.load(os.path.join(root_path, 'models', model_name)))

- Visualize for a few examples from the test set

In [ ]:
visualize_model(model, dataset, dataloader_test, target_labels, target_make_model_year_labels, num_images=10)